# Google Drive Preliminaries

# Import libraries

In [1]:
import numpy as np
from numpy.random import RandomState
import numpy.ma as ma

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
%matplotlib inline

import h5py
import ot
from numpy.random import Generator, PCG64
from sklearn import metrics
import itertools

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm
import json

In [2]:
sigAliasList    = ['sig_A', 'sig_h0', 'sig_hch', 'sig_LQ']
sigFilenameList = ['Ato4l_lepFilter_13TeV_filtered.h5', 'hToTauTau_13TeV_PU20_filtered.h5', 'hChToTauNu_13TeV_PU20_filtered.h5', 'leptoquark_LOWMASS_lepFilter_13TeV_filtered.h5']

In [3]:
#-- Set base directory and data directory path --#
basePath   = '/Users/bobli/Dropbox/AnomalyDetection/OnML4Jets2021DataChallenge/anomaly_detection_code/'
dataPath   = 'data/ADC2021/'

bkgPath    = basePath+dataPath+'background_for_training.h5'
sigPathList = []
for x in sigFilenameList:
  sigPathList.append(basePath+dataPath+x)

# Functions

In [4]:
%cd ~/Dropbox/AnomalyDetection/OnML4Jets2021DataChallenge/anomaly_detection_code/functions
%run centralFunctions.ipynb

/Users/bobli/Library/CloudStorage/Dropbox/AnomalyDetection/OnML4Jets2021DataChallenge/anomaly_detection_code/functions


# Loading Data

In [5]:
# create a dictionary to store data
dataDict = {}
dataDict['bkg'] = h5py.File(bkgPath, 'r')

for i in range(len(sigAliasList)):
  alias   = sigAliasList[i]
  sigPath = sigPathList[i]
  dataDict[alias] = h5py.File(sigPath, 'r')

In [6]:
# store data in dictionary as numpy arrays
bkg_data = dataDict['bkg']['Particles'][:, :, 0:3]
sig_data = {}

for alias in sigAliasList:
  sig_data[alias] = dataDict[alias]['Particles'][:, :, 0:3]

# kNN using distance matrix with 3d ground space

In [7]:
# set basic parameters
nEvents = 1000
random_state = Generator(PCG64(1))
OTSCHEME = {}
OTSCHEME['normPT'] = True
OTSCHEME['balanced'] = True
OTSCHEME['noZeroPad'] = False
OTSCHEME['individualOT'] = False

In [8]:
events = {}

events['bkg'] = randomDataSample(bkg_data, nEvents, random_state)

neighbor_list = list(range(5, 400,10))
avg_aucs = []
std_aucs = []
avg_ks = []
std_ks = []

i = 0

for alias in sigAliasList:
    events[alias] = randomDataSample(sig_data[alias], nEvents, random_state)

del sig_data, bkg_data

scoreDict = {}

# for each signal sample, calculate the distance matrix and perform kNN
for alias in sigAliasList:
    np.random.seed(i)
    permutation = np.random.permutation(nEvents * 2)
    
    event_list = np.concatenate((events['bkg'], events[alias]))
    event_labels = np.asarray([0] * nEvents + [1] * nEvents)
    event_list = event_list[permutation]
    event_labels = event_labels[permutation]
    
    distance_matrix = calcOTDistance(event_list, event_list, OTSCHEME, '2D', Matrix = True)
    
    avg_auc, std_auc, avg_k, std_k, metrics_list = kNN_cross_validation(distance_matrix, event_labels, neighbor_list, k_fold=5)
    print(avg_auc, std_auc, avg_k, std_k)
    avg_aucs.append(avg_auc)
    std_aucs.append(std_auc)
    avg_ks.append(avg_k)
    std_ks.append(std_k)
    scoreDict[alias] = metrics_list
    i += 1

Fitting Models: 100%|██████████| 40/40 [00:00<00:00, 63.22it/s]


0.6947582733388014 0.017948357453657943 37.0 23.15167380558045


Fitting Models: 100%|██████████| 40/40 [00:00<00:00, 62.16it/s]


0.669764010403308 0.011783244458465292 145.0 86.48699324175861


Fitting Models: 100%|██████████| 40/40 [00:00<00:00, 66.59it/s]


0.8103431594284152 0.016734256523563844 161.0 85.46344247688599


Fitting Models: 100%|██████████| 40/40 [00:00<00:00, 63.66it/s]


0.7905600332483893 0.025314221471884484 163.0 24.819347291981714


In [9]:
print(scoreDict)

{'sig_A': {'repeat0': (0.6798725411481333, array([0.        , 0.00204644, 0.00409289, 0.00613933, 0.00818578,
       0.01023222, 0.01227866, 0.01432511, 0.01637155, 0.018418  ,
       0.02046444, 0.02251089, 0.02455733, 0.02660377, 0.02916796,
       0.03425611, 0.03934426, 0.04443242, 0.04952057, 0.05460872,
       0.05969688, 0.06478503, 0.06987318, 0.07496134, 0.08004949,
       0.08513764, 0.0902258 , 0.09531395, 0.1004021 , 0.10549026,
       0.11057841, 0.11566656, 0.12075472, 0.12584287, 0.13093102,
       0.13601918, 0.14110733, 0.14619548, 0.15128364, 0.15637179,
       0.16145994, 0.1665481 , 0.17163625, 0.1767244 , 0.18181256,
       0.18690071, 0.19198886, 0.20120591, 0.21200164, 0.22279737,
       0.23359311, 0.24438884, 0.25518458, 0.26598031, 0.27677605,
       0.28757178, 0.29836751, 0.30916325, 0.31995898, 0.33075472,
       0.34155045, 0.35234619, 0.36314192, 0.37393765, 0.38473339,
       0.39552912, 0.40632486, 0.41712059, 0.42791632, 0.43871206,
       0.44950779, 

In [10]:
transposed_dict = {
    col: {row: scoreDict[row][col] for row in scoreDict}
    for col in scoreDict[next(iter(scoreDict))]
}
# Convert NumPy arrays in the lists to regular lists
converted_dict = {
    row: {
        col: [transposed_dict[row][col][0]] + [arr.tolist() for arr in transposed_dict[row][col][1:]]
        for col in transposed_dict[row]
    }
    for row in transposed_dict
}
%cd ~/Dropbox/AnomalyDetection/OnML4Jets2021DataChallenge/anomaly_detection_code/experiments/OT_ML/OT_kNN_classification/
with open('kNN_distance_matrix_2D.json', 'w') as json_file:
    json.dump(converted_dict, json_file, indent=4)

/Users/bobli/Library/CloudStorage/Dropbox/AnomalyDetection/OnML4Jets2021DataChallenge/anomaly_detection_code/experiments/OT_ML/OT_kNN_classification
